
<h1 align="center"><font size="5">COLLABORATIVE FILTERING USER-BASED</font></h1>

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from surprise import Reader, Dataset
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset, KNNBasic, SVD
from surprise.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import NearestNeighbors
from scipy.stats import pearsonr
import time as t


## Load data 

In [2]:

original_data = pd.read_csv('data_preprocessed.csv')

In [3]:

original_data.head()

,Unnamed: 0,userId,sceneId,rating
0,0,1,1,1
1,1,1,2,1
2,2,1,3,3
3,3,1,4,3
4,4,1,5,3


In [4]:
#Dropping columns  Unnamed
data = original_data.drop(['Unnamed: 0'], 1)
#data=original_data


<hr>

<a id="ref3"></a>
# Collaborative Filtering

### User-Item matrix

In [5]:

movie_matrix = data.pivot(index='userId', columns='sceneId',values='rating').fillna(0).astype(int)

In [6]:
movie_matrix


sceneId,1,2,3,4,5,6,7,8,9,10,11,12
userId,,,,,,,,,,,,
1,1,1,3,3,3,1,4,4,3,3,2,3
2,1,1,3,3,4,0,5,4,4,3,3,2
3,1,1,3,3,3,1,4,4,3,3,2,3
4,1,1,3,3,3,1,4,5,4,4,2,3
5,1,1,3,2,3,1,4,5,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
996,2,3,4,3,4,3,3,4,2,2,2,3
997,2,2,3,3,4,3,2,3,3,2,2,4
998,3,2,4,4,4,2,2,3,3,2,1,3


In [7]:
data.groupby("userId")

### Select a user x

In [8]:
selected_user=2

In [9]:
top_n_users=5

In [10]:
top_n_items=5

In [11]:
now=t.time()
inputMovies=data.groupby('userId').get_group(selected_user)

### The users who has seen the same movies as user x



In [12]:
userSubset = data[data['sceneId'].isin(inputMovies['sceneId'].tolist())]

In [13]:

userSubsetGroup = userSubset.groupby(['userId'])

In [14]:
#sort group by number of movies in common
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

### Compute similarity users to input user using Pearson Correlation


In [15]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='sceneId')
    inputMovies = inputMovies.sort_values(by='sceneId')
    nRatings = len(group)
   
    temp_df = inputMovies[inputMovies['sceneId'].isin(group['sceneId'].tolist())]
    
    tempRating = temp_df['rating'].tolist()
   
    tempGroup = group['rating'].tolist()
    
    #calculate the pearson correlation 
    Syy = sum([i**2 for i in tempGroup]) - pow(sum(tempGroup),2)/float(nRatings)
    Sxx = sum([i**2 for i in tempRating]) - pow(sum(tempRating),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRating, tempGroup)) - sum(tempRating)*sum(tempGroup)/float(nRatings)
   
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [16]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))


### The top x similar users to input user


In [17]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[1:top_n_users]
topUsers

,similarityIndex,userId
113,0.969724,114
105,0.954445,106
46,0.949134,47
114,0.943781,115


In [18]:
#merge rating_df with top users
topUsersRating=topUsers.merge(data, left_on='userId', right_on='userId', how='inner')


## Compute similarity score

multiply the movie rating by its weight (The similarity index)
,then sum up the new ratings and divide it by the sum of the weights.

In [19]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating=topUsersRating[topUsersRating['similarityIndex']>0]

In [20]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('sceneId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
#tempTopUsersRating.head()

In [21]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['prediction score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['sceneId'] = tempTopUsersRating.index
recommendation_df.head()

,prediction score,sceneId
sceneId,,
1,0.752748,1
2,0.752748,2
3,2.752748,3
4,3.000000,4
5,3.752748,5


Sort and see the top x movies that the algorithm recommended

In [22]:
recommendation_df = recommendation_df.sort_values(by='prediction score', ascending=False)
recommendation_df

,prediction score,sceneId
sceneId,,
7,4.000000,7
8,4.000000,8
5,3.752748,5
9,3.749954,9
4,3.000000,4
10,3.000000,10
11,3.000000,11
3,2.752748,3
12,2.248654,12


In [23]:
#Movies not rated by user X
movie_not_rated={}
movie_not_rated_idexes={}
for i,row in movie_matrix.iterrows():
    combine=list(zip(row.index,row.values,row))
    idx_row=[(idx,col) for idx,val,col in combine if val==0]
    indixes=[i[1] for i in idx_row]
    row_names=[i[0] for i in idx_row]
    movie_not_rated_idexes[i]=indixes
    movie_not_rated[i]=row_names


In [24]:

final_recommendation=recommendation_df.loc[recommendation_df['sceneId'].isin(movie_not_rated.get(selected_user))].sort_values(by='prediction score', ascending=False)
print("Recommendations for user {0} using user-based :".format(selected_user))
final_recommendation.head(top_n_items)
later=t.time()


Recommendations for user 2 using user-based :


In [25]:
final_recommendation.head()

,prediction score,sceneId
sceneId,,
6,0.752748,6


In [26]:
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between items
}
algo = KNNBasic(sim_options=sim_options)

In [27]:
reader = Reader(rating_scale=(1, 5))

data_df = Dataset.load_from_df( data[['userId', 'sceneId', 'rating']], reader = reader )

In [28]:
cross_validate(algo=algo, data=data_df, measures=['RMSE','MAE'], cv=5, verbose=True)
later=t.time()
print(later-now)


Computing the cosine similarity matrix...


C:\Users\Lenovo330\anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1839  1.1674  1.1691  1.1902  1.1830  1.1787  0.0089  
MAE (testset)     0.9632  0.9317  0.9297  0.9365  0.9336  0.9389  0.0123  
Fit time          0.97    1.01    0.92    0.96    0.94    0.96    0.03    
Test time         1.05    0.92    0.86    0.92    0.88    0.93    0.07    
10.77217149734497
